In [ ]:
# default_exp models.RNNPlus

# RNNPlus

> These are RNN, LSTM and GRU PyTorch implementations created by Ignacio Oguiza - timeseriesAI@gmail.com based on:

In [ ]:
#export
from tsai.imports import *
from tsai.utils import *
from tsai.data.core import *
from tsai.models.layers import *

In [ ]:
#export
class _RNN_Backbone(Module):
    def __init__(self, cell, c_in, c_out, seq_len=None, hidden_size=100, n_layers=1, bias=True, rnn_dropout=0, bidirectional=False, 
                 kss=None, include_original=True, init_weights=True):
        
        if kss:
            self.conv = MultiConcatConv1d(c_in, c_in, kss, include_original=include_original, dim=1)
            c_in = c_in * (len(kss) + include_original)
        else:
            self.conv = nn.Identity()
        rnn_layers = []
        if len(set(hidden_size)) == 1: 
            hidden_size = hidden_size[0]
            if n_layers == 1: rnn_dropout = 0
            rnn_layers.append(cell(c_in, hidden_size, num_layers=n_layers, bias=bias, batch_first=True, dropout=rnn_dropout, bidirectional=bidirectional))
            rnn_layers.append(LSTMOutput()) # this selects just the output, and discards h_n, and c_n
        else: 
            for i in range(len(hidden_size)):
                input_size = c_in if i == 0 else hs * (1 + bidirectional)
                hs = hidden_size[i] 
                rnn_layers.append(cell(input_size, hs, num_layers=1, bias=bias, batch_first=True, bidirectional=bidirectional))
                rnn_layers.append(LSTMOutput()) # this selects just the output, and discards h_n, and c_n
                if rnn_dropout and i < len(hidden_size) - 1: rnn_layers.append(nn.Dropout(rnn_dropout)) # add dropout to all layers except last
        self.rnn = nn.Sequential(*rnn_layers)
        self.transpose = Transpose(-1, -2, contiguous=True)
        if init_weights: self.apply(self._weights_init) 

    def forward(self, x):
        x = self.conv(x)                         # [batch_size x n_vars x seq_len] --> [batch_size x n_vars*len(kss) x seq_len]
        x = self.transpose(x)                    # [batch_size x n_vars x seq_len] --> [batch_size x seq_len x n_vars]
        x = self.rnn(x)                          # [batch_size x seq_len x hidden_size * (1 + bidirectional)]
        x = self.transpose(x)                    # [batch_size x hidden_size * (1 + bidirectional) x seq_len]
        print(x.shape)
        return x
    
    def _weights_init(self, m): # same init as keras
        for name, params in m.named_parameters():
            if "weight_ih" in name: 
                nn.init.xavier_normal_(params)
            elif 'weight_hh' in name: 
                nn.init.orthogonal_(params)
            elif 'bias_ih' in name:
                params.data.fill_(0)
                # Set forget-gate bias to 1
                n = params.size(0)
                params.data[(n // 4):(n // 2)].fill_(1)
            elif 'bias_hh' in name:
                params.data.fill_(0)

In [ ]:
#export
class _RNNPlus_Base(nn.Sequential):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=[100], n_layers=1, bias=True, rnn_dropout=0, bidirectional=False, 
                 fc_dropout=0., last_step=True, bn=False, custom_head=None, y_range=None, kss=None, include_original=True, init_weights=True, **kwargs):

        if not last_step: assert seq_len, 'you need to enter a seq_len to use flatten=True'

        # Backbone
        hidden_size = listify(hidden_size)
        backbone = _RNN_Backbone(self._cell, c_in, c_out, seq_len=seq_len, hidden_size=hidden_size, n_layers=n_layers,
                                 bias=bias, rnn_dropout=rnn_dropout,  bidirectional=bidirectional, kss=kss, include_original=include_original, 
                                 init_weights=init_weights)

        # Head
        self.head_nf = hidden_size * (1 + bidirectional) if isinstance(hidden_size, Integral) else hidden_size[-1] * (1 + bidirectional) 
        if custom_head: 
            if isinstance(custom_head, nn.Module): head = custom_head
            else: head = custom_head(self.head_nf, c_out, seq_len) # custom head must have all required kwargs
        else: head = self.create_head(self.head_nf, c_out, seq_len, last_step=last_step, fc_dropout=fc_dropout, bn=bn, y_range=y_range)
        super().__init__(OrderedDict([('backbone', backbone), ('head', head)]))

    def create_head(self, nf, c_out, seq_len, last_step=True, fc_dropout=0., bn=False, y_range=None):
        if last_step:
            layers = [LastStep()]
        else:
            layers = [Flatten()]
            nf *= seq_len
        if bn: layers += [nn.BatchNorm1d(nf)]
        if fc_dropout: layers += [nn.Dropout(fc_dropout)]
        layers += [nn.Linear(nf, c_out)]
        if y_range: layers += [SigmoidRange(*y_range)]
        return nn.Sequential(*layers)


class RNNPlus(_RNNPlus_Base):
    _cell = nn.RNN

class LSTMPlus(_RNNPlus_Base):
    _cell = nn.LSTM

class GRUPlus(_RNNPlus_Base):
    _cell = nn.GRU

In [ ]:
bs = 16
c_in = 3
seq_len = 12
c_out = 2
xb = torch.rand(bs, c_in, seq_len)
test_eq(RNNPlus(c_in, c_out)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, hidden_size=[100, 50, 10], bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, hidden_size=[100], n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, 
        [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(GRUPlus(c_in, c_out, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5, flatten=True)(xb).shape, 
        [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(GRUPlus(c_in, c_out, seq_len, last_step=False, flatten=True)(xb).shape, [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, hidden_size=100, n_layers=2, kss=[3,5,7], include_original=False)(xb).shape, [bs, c_out])
custom_head = nn.Sequential(Transpose(1,2), nn.Linear(8,8), nn.SELU(), nn.Linear(8, 1), Squeeze())
test_eq(LSTMPlus(c_in, c_out, hidden_size=[32,16,8,4], kss=[2,3,4], bidirectional=True,custom_head=custom_head)(xb).shape, [bs, seq_len])

In [ ]:
bs = 16
c_in = 3
seq_len = 12
c_out = 2
xb = torch.rand(bs, c_in, seq_len)
custom_head = partial(create_mlp_head, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, flatten=True, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_pool_head, concat_pool=True, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_pool_plus_head, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_conv_head)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, seq_len, hidden_size=[100, 50], n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True)(xb).shape, [bs, c_out])

In [ ]:
LSTMPlus(c_in, c_out, seq_len, hidden_size=[100, 50], n_layers=2, kss=[3,5,7], bias=True, rnn_dropout=0.2, bidirectional=True)

LSTMPlus(
  (backbone): _RNN_Backbone(
    (conv): MultiConcatConv1d(
      (layers): ModuleList(
        (0): Conv1d(3, 3, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(3, 3, kernel_size=(5,), stride=(1,), padding=(2,))
        (2): Conv1d(3, 3, kernel_size=(7,), stride=(1,), padding=(3,))
      )
    )
    (rnn): Sequential(
      (0): LSTM(12, 100, batch_first=True, bidirectional=True)
      (1): LSTMOutput()
      (2): Dropout(p=0.2, inplace=False)
      (3): LSTM(200, 50, batch_first=True, bidirectional=True)
      (4): LSTMOutput()
    )
    (transpose): Transpose(dims=-1, -2).contiguous()
  )
  (head): Sequential(
    (0): LastStep()
    (1): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [ ]:
from tsai.data.all import *
from tsai.models.utils import *
dsid = 'NATOPS' 
bs = 16
X, y, splits = get_UCR_data(dsid, return_split=False)
tfms  = [None, [Categorize()]]
dls = get_ts_dls(X, y, tfms=tfms, splits=splits, bs=bs)

In [ ]:
model = build_ts_model(LSTMPlus, dls=dls)
print(model[-1])
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): LastStep()
  (1): LinBnDrop(
    (0): Linear(in_features=100, out_features=6, bias=True)
  )
)


epoch,train_loss,valid_loss,accuracy,time
0,1.701101,1.528301,0.427778,00:01


In [ ]:
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=True)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.118912,0.611625,0.711111,00:01


In [ ]:
custom_head = partial(create_pool_head, concat_pool=True)
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=False, custom_head=custom_head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.675968,1.493234,0.605556,00:01


In [ ]:
custom_head = partial(create_pool_plus_head, concat_pool=True)
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=False, custom_head=custom_head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.871653,1.438532,0.494444,00:01


In [ ]:
m = RNNPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

RNNPlus(
  (backbone): _RNN_Backbone(
    (conv): Identity()
    (rnn): Sequential(
      (0): RNN(3, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
      (1): LSTMOutput()
    )
    (transpose): Transpose(dims=-1, -2).contiguous()
  )
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=200, out_features=2, bias=True)
    )
  )
)
(81802, True)


torch.Size([16, 2])

In [ ]:
m = LSTMPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

LSTMPlus(
  (backbone): _RNN_Backbone(
    (conv): Identity()
    (rnn): Sequential(
      (0): LSTM(3, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
      (1): LSTMOutput()
    )
    (transpose): Transpose(dims=-1, -2).contiguous()
  )
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=200, out_features=2, bias=True)
    )
  )
)
(326002, True)


torch.Size([16, 2])

In [ ]:
m = GRUPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

GRUPlus(
  (backbone): _RNN_Backbone(
    (conv): Identity()
    (rnn): Sequential(
      (0): GRU(3, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
      (1): LSTMOutput()
    )
    (transpose): Transpose(dims=-1, -2).contiguous()
  )
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=200, out_features=2, bias=True)
    )
  )
)
(244602, True)


torch.Size([16, 2])

In [ ]:
#hide
from tsai.imports import create_scripts
from tsai.export import get_nb_name
nb_name = get_nb_name()
create_scripts(nb_name);

<IPython.core.display.Javascript object>

105_models.RNNPlus.ipynb saved at 2021-11-05 06:45:42.
Converted 105_models.RNNPlus.ipynb.


Correct conversion! 😃
Total time elapsed 0.103 s
Friday 05/11/21 06:45:45 CET
